In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [12]:
def data_read(path):
    
    df = pd.read_csv(path)
    df.dropna(inplace=True)
    df.set_index('LABELS', inplace=True)
    df.index.unique()
    unwanted = ['BRICKFACE', 'FOLIAGE', 'SKY', 'WINDOW', 'PATH']
    df.drop(unwanted, inplace=True)
    
    labels_dict = {'CEMENT'  : 1,
                   'GRASS': -1}
    
    Y = np.array(df.index.map(labels_dict))
    X = np.array(df.reset_index().drop('LABELS',axis = 1))
    
    return X,Y

In [13]:
def activation(x,weights,bias):
    return np.dot(x,weights) + bias

In [14]:
def model_fit(X_train,y_train,epochs):
    shuffler = np.random.permutation(len(y_train))
    X_train = X_train[shuffler]
    y_train = y_train[shuffler]
    datapoints_len, weights_len = X_train.shape
    weights = np.zeros(weights_len)
    bias = 0
    
    weightsEpoch = np.zeros((epochs,weights_len))
    biasEpoch = np.zeros(epochs)
    
    averagedWeight = weights
    averagedBias = bias
    
    for epoch in range(epochs):
        for feature in range(datapoints_len):
            x = X_train[feature]
            y = y_train[feature]
            a = activation(x, weights, bias)

            if y*a<=0:
                weights = weights + y*x
                bias += y
            
            iteration = (epoch+1)*(feature+1)
            averagedWeight = averagedWeight*((iteration-1)/iteration) + (weights/iteration)
            averagedBias = averagedBias*((iteration-1)/iteration) + (bias/iteration)

        
        weightsEpoch[epoch] = averagedWeight
        biasEpoch[epoch] = averagedBias
                
    return averagedWeight, averagedBias, weightsEpoch, biasEpoch

In [15]:
def accuracy(X, Y, weights, bias):
    y_predicted = list(map(lambda i: np.sign(activation(i,weights,bias)),X))
    accuracy = np.sum(y_predicted*Y)/len(y_predicted*Y)
    return accuracy


In [16]:
X_train,y_train = data_read("segmentation-train.csv")
epochs = 40
weights, bias, weightsEpoch, biasEpoch = model_fit(X_train,y_train,epochs)

In [17]:
train_accuracy = accuracy(X_train, y_train, weights, bias)
train_accuracy

1.0

In [18]:
X_test,y_test = data_read("segmentation-test.csv")
test_accuracy = accuracy(X_test, y_test, weights, bias)
test_accuracy

0.9833333333333333

In [19]:
accuracyEpochWise = [accuracy(X_test, y_test, weightsEpoch[i], biasEpoch[i]) for i in range(epochs)]

In [ ]:
plt.plot(np.arange(1,epochs+1),accuracyEpochWise,'-b')
plt.xlabel("epochs")
plt.ylabel("accuracy")

Text(0, 0.5, 'accuracy')